# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
import cv2
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm
import os
import pickle
import copy
from skimage import exposure
from PIL import Image
from PIL import ImageOps
from sklearn import preprocessing

In [2]:
# TODO: Fill this in based on where you saved the training and testing data
filePath = 'traffic-signs-data/'
training_file = filePath+'train.p'
validation_file= filePath+'valid.p'
testing_file = filePath+'test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_validation = X_valid.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of an traffic sign image?
image_shape = (X_train.shape[1], X_train.shape[2])

# TODO: How many unique classes/labels there are in the dataset.
labels = pd.read_csv('signnames.csv')
n_classes = len(labels)

print("Number of training examples: x={0}, y={1}".format(n_train, y_train.shape[0]))
print("Number of validation examples: x={0}, y={1}".format(n_validation, y_valid.shape[0]))
print("Number of testing examples:  x={0}, y={1}".format(n_test, y_test.shape[0]))
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

In [ ]:
y_train_df = pd.DataFrame(data=y_train, columns=['ClassId'])
y_valid_df = pd.DataFrame(data=y_valid, columns=['ClassId'])
y_test_df = pd.DataFrame(data=y_test, columns=['ClassId'])

grouped_train_index = y_train_df.groupby('ClassId')
grouped_valid_index = y_valid_df.groupby('ClassId')
grouped_test_index = y_test_df.groupby('ClassId')

## Get the first index images in each class
firstTrainDataForEachClass = grouped_train_index.head(1).sort_values('ClassId').reset_index()
#print(firstTrainDataForEachClass)

### Pre-process the Data Set (normalization, grayscale, etc.)

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

## Ganussian Filter
def gaussianFilter(img):
    # Gaussian filter
    filter_size = 3
    filter_sigma = 0.2
    gaussian_filter = np.dot(cv2.getGaussianKernel(filter_size, 
                                                   filter_sigma, 
                                                   cv2.CV_64F),
                            (cv2.getGaussianKernel(filter_size, 
                                                   filter_sigma, 
                                                   cv2.CV_64F)).T)

    img_filtered = cv2.filter2D(img, -1, 
                                gaussian_filter, 
                                borderType=cv2.BORDER_REFLECT_101) # reflect across edge filter
    
    return img_filtered

## Change to gray scale
def colourToGrey(images):
    gray = np.zeros([len(images), 32, 32], dtype=np.float64)
    for idx in tqdm(range(len(images))):
        gray[idx] = ImageOps.grayscale(Image.fromarray(images[idx]))
    return gray

## Normalisation
def normaliseGrayImages(images):
    norm = copy.copy(images)
    
    for idx, img in tqdm(enumerate(images)):
        norm[idx] = preprocessing.normalize(norm[idx], norm='l2')
    return norm

## Enhance Contrast using histogram equalization
def enhanceContrastOfImage(img):
    return cv2.equalizeHist(img)

def sharpenImage(img):
    kernelOfSharpener = np.array([[-1,-1,-1,-1,-1], 
                                 [-1,2,2,2,-1], 
                                 [-1,2,8,2,-1],
                                 [-1,2,2,2,-1],
                                 [-1,-1,-1,-1,-1]]) / 8.0
    return cv2.filter2D(img, -1, kernelOfSharpener)

def affineTransformation(img):
    row, column = X_train[8000].shape[:2]
    sourcePoints = np.float32([[0, 0], [column-1, 0], [0, row-1]])

    destinationPoints = np.float32([[np.random.uniform(low=0., high=0.2, size=1)[0]*(row-1), np.random.uniform(low=0., high=0.2, size=1)[0]*(column-1)], 
                                    [np.random.uniform(low=0.7, high=1, size=1)[0]*(row-1), np.random.uniform(low=0., high=0.2, size=1)[0]*(column-1)], 
                                    [np.random.uniform(low=0., high=0.2, size=1)[0]*(row-1), np.random.uniform(low=0.7, high=1, size=1)[0]*(column-1)]])

    affine_matrix = cv2.getAffineTransform(sourcePoints, destinationPoints)
    #img_output = cv2.warpAffine(X_train[8000], affine_matrix, (row, column))
    
    return cv2.warpAffine(img, affine_matrix, (row, column))

def translateImageWithRandomDistance(img):
    row, column = img.shape[:2]
    matrixForTranslation = np.array([ [1, 0, np.random.randint(low=-5, high=5)], [0, 1, np.random.randint(low=-5, high=5)] ], dtype=float)
    translatedImage = cv2.warpAffine(img, matrixForTranslation, (row, column))
    return translatedImage

def rotateImageWithRandomAngle(img):
    row, column = img.shape[:2]
    matrixForRotation = cv2.getRotationMatrix2D((column/2, row/2), np.random.randint(low=-35, high=35), 1)
    rotatedImage = cv2.warpAffine(img, matrixForRotation, (row, column))
    return rotatedImage

def makeBallanceBetweenClasses(grouped_y_index=None, xData=None, yData=None):
    ## balance the number of samples in classes
    maxSampleNumbersAmongClasses = max(grouped_y_index.size())
    argmaxSampleNumbersAmongClasses = np.argmax(grouped_y_index.size())

    for class_idx, value in tqdm(grouped_y_index.groups.items()):
        '''
        if class_idx>1:
            break
        '''
        necessaryNum = maxSampleNumbersAmongClasses - len(value)
        randomImgInAClass = xData[np.random.choice(grouped_y_index.groups[class_idx].values, necessaryNum)]

        augmented_y = np.ones(necessaryNum) * class_idx

        for idx, img in enumerate(randomImgInAClass):
            randomImgInAClass[idx] = transAndRotate(img)
            #randomImgInAClass[idx] = transAndRotate(colourToGrey(img))
        xData = np.vstack([xData, randomImgInAClass])
        yData = np.append(yData, augmented_y)
        
    return xData, yData

def transAndRotate(img):
    switch = np.random.choice(('affine', 'rotate', 'translate', 'rotate+translate'), size=1)[0] # choose one among three strings
    #print(switch)
    if switch == 'affine':
        return affineTransformation(img)
    if switch == 'rotate':
        return rotateImageWithRandomAngle(img)
    if switch == 'translate':
        return translateImageWithRandomDistance(img)
    if switch == 'rotate+translate':
        return translateImageWithRandomDistance(img)

In [3]:
def saveToPickle(data, file_name=None, folder_path=None):
    if not os.path.isdir(folder_path):
        #print("Create \"preprocessed-data\" folder")
        os.mkdir(folder_path)
    else:
        print("\"preprocessed-data\" folder already exist")

    file_name = folder_path + file_name
    if not os.path.exists(file_name):
        try:
            with open(file_name, 'wb') as f:
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
                print("Create", file_name)
        except Exception as e:
            print('Error: unable to save data to', file_name, 'because', e)
            
def loadPickle(file_name=None, folder_path=None):
    file = folder_path + file_name
    #print('Load')
    if os.path.exists(file):
        try:
            with open(file, 'rb') as f:
                return pickle.load(f)
                print("Open", file)
        except Exception as e:
            print('Error: unable to open data to', file, 'because', e)

#### Using PIL

In [ ]:
# colour to gray
augmented_X_train = colourToGrey(X_train)
augmented_X_valid = colourToGrey(X_valid)
augmented_X_test = colourToGrey(X_test)

In [ ]:
plt.imshow(augmented_X_train[7809], cmap='gray')
print(labels.loc[y_train[7809]])

In [ ]:
augmented_X_train = sharpenImage(augmented_X_train)
augmented_X_valid = sharpenImage(augmented_X_valid)
augmented_X_test = sharpenImage(augmented_X_test)

In [ ]:
plt.imshow(augmented_X_train[7809], cmap='gray')
print(labels.loc[y_train[7809]])

In [ ]:
# normalisation
augmented_X_train = normaliseGrayImages(augmented_X_train)
augmented_X_valid = normaliseGrayImages(augmented_X_valid)
augmented_X_test = normaliseGrayImages(augmented_X_test)

In [ ]:
plt.imshow(augmented_X_train[6809], cmap='gray')
print(labels.loc[y_train[6809]])

In [ ]:
# Apply localized histogram localization  
for i in tqdm(range(augmented_X_train.shape[0])):
    augmented_X_train[i] = exposure.equalize_adapthist(augmented_X_train[i])
    
for i in tqdm(range(augmented_X_valid.shape[0])):
    augmented_X_valid[i] = exposure.equalize_adapthist(augmented_X_valid[i])
    
for i in tqdm(range(augmented_X_test.shape[0])):
    augmented_X_test[i] = exposure.equalize_adapthist(augmented_X_test[i])

In [ ]:
augmented_X_train, augmented_y_train = makeBallanceBetweenClasses(grouped_train_index, augmented_X_train, y_train)
augmented_X_valid, augmented_y_valid = makeBallanceBetweenClasses(grouped_valid_index, augmented_X_valid, y_valid)
augmented_X_test, augmented_y_test = makeBallanceBetweenClasses(grouped_test_index, augmented_X_test, y_test)

In [ ]:
plt.imshow(augmented_X_valid[120], cmap='gray')
print(labels.loc[augmented_y_valid[120]])

In [ ]:
saveToPickle(augmented_X_train, file_name='augmented_X_train.p', folder_path="./preprocessed-data/")
saveToPickle(augmented_y_train, file_name='augmented_y_train.p', folder_path="./preprocessed-data/")

saveToPickle(augmented_X_valid, file_name='augmented_X_valid.p', folder_path="./preprocessed-data/")
saveToPickle(augmented_y_valid, file_name='augmented_y_valid.p', folder_path="./preprocessed-data/")

saveToPickle(augmented_X_test, file_name='augmented_X_test.p', folder_path="./preprocessed-data/")
saveToPickle(augmented_y_test, file_name='augmented_y_test.p', folder_path="./preprocessed-data/")

In [4]:
## Load
augmented_X_train = loadPickle(file_name='augmented_X_train.p', folder_path='./preprocessed-data/')
augmented_y_train = loadPickle(file_name='augmented_y_train.p', folder_path='./preprocessed-data/')

augmented_X_valid = loadPickle(file_name='augmented_X_valid.p', folder_path='./preprocessed-data/')
augmented_y_valid = loadPickle(file_name='augmented_y_valid.p', folder_path='./preprocessed-data/')

augmented_X_test = loadPickle(file_name='augmented_X_test.p', folder_path='./preprocessed-data/')
augmented_y_test = loadPickle(file_name='augmented_y_test.p', folder_path='./preprocessed-data/')

print('The shape of the loaded processed X train dataset:', augmented_X_train.shape)
print('The shape of the loaded processed y train dataset:', augmented_y_train.shape)

print('The shape of the loaded processed X valid dataset:', augmented_X_valid.shape)
print('The shape of the loaded processed y valid dataset:', augmented_y_valid.shape)

print('The shape of the loaded processed X test dataset:', augmented_X_test.shape)
print('The shape of the loaded processed y test dataset:', augmented_y_test.shape)

The shape of the loaded processed X train dataset: (86430, 32, 32, 1)
The shape of the loaded processed y train dataset: (86430,)
The shape of the loaded processed X valid dataset: (10320, 32, 32, 1)
The shape of the loaded processed y valid dataset: (10320,)
The shape of the loaded processed X test dataset: (32250, 32, 32, 1)
The shape of the loaded processed y test dataset: (32250,)


#### Plot histograms

In [ ]:
plt.hist(X_train[34000].ravel(), bins=256, histtype='step', color='black')
plt.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
cdf = plt.twinx()

In [ ]:
plt.hist(augmented_X_train[34000].ravel(), bins=256, histtype='step', color='black')
plt.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
cdf = plt.twinx()
img_cdf, bins = exposure.cumulative_distribution(augmented_X_train[0], 256)
cdf.plot(bins, img_cdf, 'r')

---

---

### Batches

In [5]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    if batch_size > 0:
        assert len(features) == len(labels)

        output_batches = []
        sample_size = len(features)

        for start_i in range(0, sample_size, batch_size):
            end_i = start_i + batch_size
            batch = [features[start_i:end_i], labels[start_i:end_i]]
            output_batches.append(batch)
    else:
        assert len(features) == len(labels)
        
        output_batches = []
        sample_size = len(features)
        
        for start in range(sample_size):
            batch = [features[start], labels[start]]
            output_batches.append(batch)
            
    return output_batches

### Model Architecture

In [6]:
rate = 0.001
EPOCHS = 30
#BATCH_SIZE = 128
BATCH_SIZE = 64
DROPOUT = 0.5

In [7]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    ## Layer 1
    # Convolutional. Input = 32x32x1. Output = 32x32x32.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(1, 1, 1, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
    # Activation.
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.dropout(conv1, 0.6)
    
    # Pooling. Input = 32x32x32. Output = 16x16x32.
    conv1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    ## Layer 2
    # Convolutional. Input = 16x16x32. Output = 14x14x96.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 32, 96), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(96))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b

    # Activation.
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.dropout(conv2, 0.6)

    # Pooling. Input = 14x14x96. Output = 7x7x96.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    ## Layer 3 
    #Convolutional. Input = 7x7x96. Output = 5x5x288.
    conv3_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 96, 288), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(288))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='VALID') + conv3_b
    
    # Activation.
    conv3 = tf.nn.relu(conv3)
    conv3 = tf.nn.dropout(conv3, 0.6)

    # Pooling. Input = 5x5x288. Output = 3x3x288.
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 3x3x288. Output = 1152.
    fc0   = flatten(conv3)
    
    # Layer 3: Fully Connected. Input = 1152. Output = 1152.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(1152, 1152), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(1152))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    fc1    = tf.nn.relu(fc1)
    # fc1 = tf.nn.dropout(fc1, 0.6)

    # Layer 4: Fully Connected. Input = 1152. Output = 576.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(1152, 576), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(576))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Activation.
    fc2    = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, 0.6)

    # Layer 5: Fully Connected. Input = 576. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(576, 43), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [8]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

## Features and Labels
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

In [9]:
## Training Pipeline
logits =  LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [10]:
## Model Evaluation
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data, batch_size, sess):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()

    for batch_x, batch_y in tqdm(batches(batch_size, X_data, y_data)):
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [11]:
## Reshape dataset
augmented_X_train = np.reshape(augmented_X_train, (len(augmented_X_train), 32, 32, 1))
augmented_X_valid = np.reshape(augmented_X_valid, (len(augmented_X_valid), 32, 32, 1))
augmented_X_test = np.reshape(augmented_X_test, (len(augmented_X_test), 32, 32, 1))
print(augmented_X_train[:128].shape)

(128, 32, 32, 1)


In [12]:
print(len(augmented_X_train))
print(len(augmented_X_valid))

86430
10320


In [13]:
## Train the Model

with tf.Session() as session:
    session.run(tf. global_variables_initializer())
    num_examples = len(augmented_X_train)
    
    print("Training...")
    #print()
    for i in range(EPOCHS):
        shuffle_X, shuffle_y = shuffle(augmented_X_train, augmented_y_train)
        for batch_x, batch_y in tqdm(batches(BATCH_SIZE, shuffle_X, shuffle_y)):
            session.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: DROPOUT})
        
        shuffle_X_valid, shuffle_y_valid = shuffle(augmented_X_valid, augmented_y_valid)
        validation_accuracy = evaluate(shuffle_X_valid, shuffle_y_valid, BATCH_SIZE, session)
        print("EPOCH {0} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
    
    saver.save(session, './weights')
    print("Model Saved")

Training...


100%|███████████████████████████████████████| 162/162 [00:01<00:00, 109.39it/s]


EPOCH 1 ...
Validation Accuracy = 0.667



100%|███████████████████████████████████████| 162/162 [00:01<00:00, 107.28it/s]


EPOCH 2 ...
Validation Accuracy = 0.725



...

EPOCH 28 ...
Validation Accuracy = 0.818



100%|███████████████████████████████████████| 162/162 [00:01<00:00, 109.16it/s]


EPOCH 29 ...
Validation Accuracy = 0.822



100%|███████████████████████████████████████| 162/162 [00:01<00:00, 111.03it/s]


EPOCH 30 ...
Validation Accuracy = 0.821

Model Saved


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [1]:
# Launch the graph
#saver = tf.train.Saver()
with tf.Session() as session:
    saver.restore(session, tf.train.latest_checkpoint('./'))
    '''
    new_saver = tf.train.import_meta_graph("weights.meta")
    new_saver.restore(session, tf.train.latest_checkpoint('./'))
    all_vars = tf.get_collection('vars')
    for v in all_vars:
        v_ = session.run(v)
        print(v_)
    '''
    saver.restore(session, './weights')
    accuracy = evaluate(augmented_X_test, augmented_y_test, BATCH_SIZE, session)
    print(accuracy)

NameError: name 'tf' is not defined

### Predict the Sign Type for Each Image

In [ ]:
newTestImages = X_test[randomIndex] # copy X_test for 5 images

''' Colour
img1 = cv2.cvtColor(cv2.imread('Test_images/no passing.jpg'), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread('Test_images/right of way at the next intersection.jpg'), cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(cv2.imread('Test_images/speed limit 80km.jpg'), cv2.COLOR_BGR2RGB)
img4 = cv2.cvtColor(cv2.imread('Test_images/yield.jpg'), cv2.COLOR_BGR2RGB)
img5 = cv2.cvtColor(cv2.imread('Test_images/general causion.png'), cv2.COLOR_BGR2RGB)
'''

## Gray
img1 = cv2.cvtColor(cv2.imread('./Images/Test_images/no passing.jpg'), cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(cv2.imread('./Images/Test_images/right of way at the next intersection.jpg'), cv2.COLOR_BGR2GRAY)
img3 = cv2.cvtColor(cv2.imread('./Images/Test_images/speed limit 80km.jpg'), cv2.COLOR_BGR2GRAY)
img4 = cv2.cvtColor(cv2.imread('./Images/Test_images/yield.jpg'), cv2.COLOR_BGR2GRAY)
img5 = cv2.cvtColor(cv2.imread('./Images/Test_images/general causion.png'), cv2.COLOR_BGR2GRAY)

newTestImages[0] = cv2.resize(img1, (32, 32))
newTestImages[1] = cv2.resize(img2, (32, 32)) 
newTestImages[2] = cv2.resize(img3, (32, 32)) 
newTestImages[3] = cv2.resize(img4, (32, 32)) 
newTestImages[4] = cv2.resize(img5, (32, 32)) 

fig, ax = plt.subplots(5, 1, figsize=[10, 10])

newTestImages = np.reshape(newTestImages, (len(newTestImages), 32, 32, 1)) # Gray

with tf.Session() as session:
    session.run(tf. global_variables_initializer())
    saver.restore(session, './weights')

    output = session.run(logits, feed_dict={x: newTestImages, keep_prob: 1})

correct_index = [9, 11, 5, 13, 18]
correctLabels = labels.ix[correct_index]['SignName'].values

predict_index = output.argmax(axis=1)
predictLabels = labels.ix[predict_index]['SignName'].values

ax[0].imshow(img1)
ax[0].set_title('Correct: '+correctLabels[0]+' -> Predict: '+predictLabels[0])

ax[1].imshow(img2)
ax[1].set_title('Correct: '+correctLabels[1]+' -> Predict: '+predictLabels[1])

ax[2].imshow(img3)
ax[2].set_title('False: '+correctLabels[2]+' -> Predict: '+predictLabels[2])

ax[3].imshow(img4)
ax[3].set_title('Correct: '+correctLabels[3]+' -> Predict: '+predictLabels[3])

ax[4].imshow(img5)
ax[4].set_title('Correct: '+correctLabels[4]+' -> Predict: '+predictLabels[4])

plt.tight_layout()

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.
# Launch the graph

def testEvaluate(X_data, y_data, sess):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    
    assert len(X_data) == len(y_data)
    
    for idx in range(len(y_data)):
        accuracy = sess.run(accuracy_operation, feed_dict={x: X_data, y: y_data, keep_prob: 1})
        total_accuracy += accuracy
    return total_accuracy / num_examples

with tf.Session() as session:
    session.run(tf. global_variables_initializer())
    saver.restore(session, './weights')
    
    accuracyOnTest = testEvaluate(newTestImages, correct_index, session)
    
print(accuracyOnTest)

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.
with tf.Session() as session:
    FivesoftmaxProbabilities = session.run(tf.nn.top_k(tf.constant(output), sorted=True, k=5))

print('5 Softmax Probabilities For Each Image:\n', FivesoftmaxProbabilities[1])

In [ ]:
first = []
for idx in FivesoftmaxProbabilities[1][0]:
    first = np.append(first, firstTrainDataForEachClass.iloc[idx].values[0])
first = first.astype(int)

print('Label for the sampled image:', correctLabels[0])

fig, ax = plt.subplots(1, 5, figsize=[20, 20])

sns.set_style(style='white')
for idx, img_idx in enumerate(first):
    ax[idx].imshow(sharpenImage(enhanceContrastOfImage(X_train[img_idx])))
    label = labels.loc[FivesoftmaxProbabilities[1][0][idx]]
    ax[idx].set_title(label[1], fontsize=16)
plt.tight_layout()

In [ ]:
first = []
for idx in FivesoftmaxProbabilities[1][1]:
    first = np.append(first, firstTrainDataForEachClass.iloc[idx].values[0])
first = first.astype(int)

print('Label for the sampled image:', correctLabels[1])

fig, ax = plt.subplots(1, 5, figsize=[20, 20])

sns.set_style(style='white')
for idx, img_idx in enumerate(first):
    ax[idx].imshow(sharpenImage(enhanceContrastOfImage(X_train[img_idx])))
    label = labels.loc[FivesoftmaxProbabilities[1][1][idx]]
    ax[idx].set_title(label[1], fontsize=16)
plt.tight_layout()

In [ ]:
first = []
for idx in FivesoftmaxProbabilities[1][2]:
    first = np.append(first, firstTrainDataForEachClass.iloc[idx].values[0])
first = first.astype(int)

print('Label for the sampled image:', correctLabels[2])

fig, ax = plt.subplots(1, 5, figsize=[20, 20])

sns.set_style(style='white')
for idx, img_idx in enumerate(first):
    ax[idx].imshow(sharpenImage(enhanceContrastOfImage(X_train[img_idx])))
    label = labels.loc[FivesoftmaxProbabilities[1][2][idx]]
    ax[idx].set_title(label[1], fontsize=20)
plt.tight_layout()

In [ ]:
first = []
for idx in FivesoftmaxProbabilities[1][3]:
    first = np.append(first, firstTrainDataForEachClass.iloc[idx].values[0])
first = first.astype(int)

print('Label for the sampled image:', correctLabels[3])

fig, ax = plt.subplots(1, 5, figsize=[20, 20])

sns.set_style(style='white')
for idx, img_idx in enumerate(first):
    ax[idx].imshow(sharpenImage(enhanceContrastOfImage(X_train[img_idx])))
    label = labels.loc[FivesoftmaxProbabilities[1][3][idx]]
    ax[idx].set_title(label[1], fontsize=20)
plt.tight_layout()

In [ ]:
first = []
for idx in FivesoftmaxProbabilities[1][4]:
    first = np.append(first, firstTrainDataForEachClass.iloc[idx].values[0])
first = first.astype(int)

print('Label for the sampled image:', correctLabels[4])

fig, ax = plt.subplots(1, 5, figsize=[20, 20])

sns.set_style(style='white')
for idx, img_idx in enumerate(first):
    ax[idx].imshow(sharpenImage(enhanceContrastOfImage(X_train[img_idx])))
    label = labels.loc[FivesoftmaxProbabilities[1][4][idx]]
    ax[idx].set_title(label[1], fontsize=20)
plt.tight_layout()

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")